<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/master/crossValidation/ModelA%26Strided-CNN-A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model.py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 256
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-22 19:19:10--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-22 19:19:10 (121 MB/s) - ‘utils.py’ saved [4519/4519]



In [0]:
import numpy as np
np.random.seed(1)
indexSet=np.random.permutation(50000)
prob=0.8
indexTrain=indexSet[0:int(50000*prob)]
indexValid=indexSet[int(50000*prob):-1]

In [0]:
cuda = cuda and torch.cuda.is_available()
CIFAR10_Train = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = CIFAR10_Train.data[indexTrain].mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = CIFAR10_Train.data[indexTrain].std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


In [0]:
del CIFAR10_Train
CIFAR10_Train=datasets.CIFAR10( root=dataset_path, train=True, download=True, transform=transform_train)
teainSet=torch.utils.data.Subset(CIFAR10_Train,indexTrain)
validSet=torch.utils.data.Subset(CIFAR10_Train,indexValid)

Files already downloaded and verified


In [0]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(teainSet,
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(validSet,
    batch_size=test_batch_size, shuffle=False, **kwargs)

In [0]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
CIFAR10_test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
global_loss=100000000.0
global_acc=0
global_model_path=''

In [0]:
def test(epoch, best_loss, best_epoch, lr,global_loss,global_acc,global_model_path):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best_"+str(lr)+".pt")
        try:
          if gsync_save:
              gsync.update_file_to_folder("best_"+str(lr)+".pt")
        except:
          print('Failed to gsync_save.')

        # Save the best model among three different lr
        if test_loss<global_loss:

          try:
            model_save_name = 'best_'+str(lr)+'_'+str(np.where(model.baseModel))+'_'+str(np.where(model.modifiedModel))+'_'
            path = F"/content/drive/My Drive/dl-reproducibility-project/model/{model_save_name}" 
            torch.save(model.state_dict(), path+'.epoch-{}.pt'.format(epoch))
          except:
            print('Failed to save best model to personal google drive')

          global_acc=correct
          global_loss=test_loss
          try:
            os.remove(global_model_path)
          except:
            print('Failed to delete the file')
          global_model_path=global_model_path= path+'.epoch-{}.pt'.format(epoch)
          
    return best_loss, best_epoch, correct,global_loss,global_acc,global_model_path

In [0]:
def evaluate():
    model.eval()

    prediction=[]
    test_loss = 0
    correct = 0

    for data, target in CIFAR10_test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        prediction+=[output]
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(CIFAR10_test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(CIFAR10_test_loader.dataset), 100. * correct /
            len(CIFAR10_test_loader.dataset)))
    
   

    return test_loss, correct, prediction

In [0]:
import os
try:
  os.makedirs('/content/drive/My Drive/dl-reproducibility-project/model/')
except:
  print('')

Model A without modification

In [0]:
for lr in [0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model()
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch, acc,global_loss,global_acc,global_model_path = test(epoch, best_loss, best_epoch, lr,global_loss,global_acc,global_model_path)
      if (epoch>100)&(acc.item()>0)&((acc.item()/len(test_loader.dataset))<=0.1):
        print("Stop at acc of test set:",acc.item()/len(test_loader.dataset))
        break

Training with  0.1


Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Dropout(p=0.5, inplace=False)
    (9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (14): ReLU()
    (15): AdaptiveAvgPool2d(output_size=1)
    (16): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/40000 (0%)]	Loss: 2.302207
Train Epoch: 0 [25600/40000 (64%)]	Loss: 2.014185

Test set: Average loss: 0.0153, Accuracy: 2848/9999 (28%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:00<00:00, 102.03it/s]


Failed to delete the file
Train Epoch: 1 [0/40000 (0%)]	Loss: 1.999851
Train Epoch: 1 [25600/40000 (64%)]	Loss: 1.733370

Test set: Average loss: 0.0144, Accuracy: 3589/9999 (36%)



...

Train Epoch: 114 [0/40000 (0%)]	Loss: 0.538395
Train Epoch: 114 [25600/40000 (64%)]	Loss: 0.494439

Test set: Average loss: 0.0059, Accuracy: 7486/9999 (75%)

Train Epoch: 115 [0/40000 (0%)]	Loss: 0.626798
Train Epoch: 115 [25600/40000 (64%)]	Loss: 0.558941

Test set: Average loss: 0.0055, Accuracy: 7620/9999 (76%)

Train Epoch: 116 [0/40000 (0%)]	Loss: 0.626121
Train Epoch: 116 [25600/40000 (64%)]	Loss: 0.673197

Test set: Average loss: 0.0056, Accuracy: 7632/9999 (76%)

Train Epoch: 117 [0/40000 (0%)]	Loss: 0.558939
Train Epoch: 117 [25600/40000 (64%)]	Loss: 0.629800

Test set: Average loss: 0.0057, Accuracy: 7579/9999 (76%)

Train Epoch: 118 [0/40000 (0%)]	Loss: 0.635341
Train Epoch: 118 [25600/40000 (64%)]	Loss: 0.542639

Test set: Average loss: 0.0056, Accuracy: 7599/9999 (76%)

Train Epoch: 119 [0/40000 (0%)]	Loss: 0.574920
Train Epoch: 119 [25600/40000 (64%)]	Loss: 0.702478

Test set: Average loss: 0.0054, Accuracy: 7705/9999 (77%)

Train Epoch: 120 [0/40000 (0%)]	Loss: 0.616593

Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:01<00:00, 77.04it/s]


Train Epoch: 147 [0/40000 (0%)]	Loss: 0.483122
Train Epoch: 147 [25600/40000 (64%)]	Loss: 0.496991

Test set: Average loss: 0.0055, Accuracy: 7734/9999 (77%)

Train Epoch: 148 [0/40000 (0%)]	Loss: 0.567895
Train Epoch: 148 [25600/40000 (64%)]	Loss: 0.545068

Test set: Average loss: 0.0055, Accuracy: 7648/9999 (76%)

Train Epoch: 149 [0/40000 (0%)]	Loss: 0.566361
Train Epoch: 149 [25600/40000 (64%)]	Loss: 0.578773

Test set: Average loss: 0.0058, Accuracy: 7543/9999 (75%)

Train Epoch: 150 [0/40000 (0%)]	Loss: 0.622184
Train Epoch: 150 [25600/40000 (64%)]	Loss: 0.472800

Test set: Average loss: 0.0053, Accuracy: 7763/9999 (78%)

Train Epoch: 151 [0/40000 (0%)]	Loss: 0.553421
Train Epoch: 151 [25600/40000 (64%)]	Loss: 0.666307

Test set: Average loss: 0.0055, Accuracy: 7697/9999 (77%)

Train Epoch: 152 [0/40000 (0%)]	Loss: 0.462448
Train Epoch: 152 [25600/40000 (64%)]	Loss: 0.550126

Test set: Average loss: 0.0055, Accuracy: 7731/9999 (77%)

Train Epoch: 153 [0/40000 (0%)]	Loss: 0.471735

Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:01<00:00, 68.50it/s]


Train Epoch: 179 [0/40000 (0%)]	Loss: 0.596096
Train Epoch: 179 [25600/40000 (64%)]	Loss: 0.422918

Test set: Average loss: 0.0055, Accuracy: 7709/9999 (77%)

Train Epoch: 180 [0/40000 (0%)]	Loss: 0.582244
Train Epoch: 180 [25600/40000 (64%)]	Loss: 0.459308

Test set: Average loss: 0.0056, Accuracy: 7680/9999 (77%)

Train Epoch: 181 [0/40000 (0%)]	Loss: 0.416241
Train Epoch: 181 [25600/40000 (64%)]	Loss: 0.501971

Test set: Average loss: 0.0055, Accuracy: 7709/9999 (77%)

Train Epoch: 182 [0/40000 (0%)]	Loss: 0.525230
Train Epoch: 182 [25600/40000 (64%)]	Loss: 0.498179

Test set: Average loss: 0.0055, Accuracy: 7737/9999 (77%)

Train Epoch: 183 [0/40000 (0%)]	Loss: 0.547015
Train Epoch: 183 [25600/40000 (64%)]	Loss: 0.457740

Test set: Average loss: 0.0055, Accuracy: 7741/9999 (77%)

Train Epoch: 184 [0/40000 (0%)]	Loss: 0.543731
Train Epoch: 184 [25600/40000 (64%)]	Loss: 0.468906

Test set: Average loss: 0.0054, Accuracy: 7793/9999 (78%)

Train Epoch: 185 [0/40000 (0%)]	Loss: 0.509932

In [0]:
try: 
  del model
except:
  print('')

# Maybe it's redundant 
try:
  del criterion
  del optimizer
  del scheduler
except:
  print('')

try:
  torch.cuda.empty_cache()
except:
  print('')

#
# Chnage Here
#
# Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
# Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
# The scecond one is the strided. The third: ConvPool. The last: All CNN.
model = Model()
model.load_state_dict(torch.load(global_model_path))
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
test_loss, correct, prediction=evaluate()


Test set: Average loss: 0.0051, Accuracy: 7955/10000 (80%)



In [0]:
print('Validation Set')
print('Error: ',global_loss.item())
print('Acc',global_acc.item()/len(indexValid))
print('Model: ',global_model_path)

Validation Set
Error:  0.005021385848522186
Acc 0.7880788078807881
Model:  /content/drive/My Drive/dl-reproducibility-project/model/best_0.01.epoch-178.pt


# Strided-CNN-A


In [0]:
modifiedModel=[False, True, False, False]

for lr in [0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model(modifiedModel=modifiedModel)
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch, acc,global_loss,global_acc,global_model_path = test(epoch, best_loss, best_epoch, lr,global_loss,global_acc,global_model_path)
      if (epoch>100)&(acc.item()>0)&((acc.item()/len(test_loader.dataset))<=0.1):
        print("Stop at acc of test set:",acc.item()/len(test_loader.dataset))
        break
try: 
  del model
except:
  print('')

# Maybe it's redundant 
try:
  del criterion
  del optimizer
  del scheduler
except:
  print('')

try:
  torch.cuda.empty_cache()
except:
  print('')

#
# Chnage Here
#
# Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
# Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
# The scecond one is the strided. The third: ConvPool. The last: All CNN.
model = Model(modifiedModel=modifiedModel)
model.load_state_dict(torch.load(global_model_path))
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
test_loss, correct, prediction=evaluate()

print('Validation Set')
print('Error: ',global_loss.item())
print('Acc',global_acc.item()/len(indexValid))
print('Model: ',global_model_path)

Training with  0.1


Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv2d(96, 192, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): AdaptiveAvgPool2d(output_size=1)
    (14): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (conv_1_192_class): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
  (BN_96): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN_19

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/40000 (0%)]	Loss: 2.305290
Train Epoch: 0 [25600/40000 (64%)]	Loss: 1.953498

Test set: Average loss: 0.0152, Accuracy: 2632/9999 (26%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:00<00:00, 109.51it/s]


Failed to delete the file
Train Epoch: 1 [0/40000 (0%)]	Loss: 1.843767
Train Epoch: 1 [25600/40000 (64%)]	Loss: 1.722043

Test set: Average loss: 0.0133, Accuracy: 3829/9999 (38%)



...

Train Epoch: 141 [0/40000 (0%)]	Loss: 0.616835
Train Epoch: 141 [25600/40000 (64%)]	Loss: 0.610169

Test set: Average loss: 0.0057, Accuracy: 7724/9999 (77%)

Train Epoch: 142 [0/40000 (0%)]	Loss: 0.530167
Train Epoch: 142 [25600/40000 (64%)]	Loss: 0.503581

Test set: Average loss: 0.0057, Accuracy: 7688/9999 (77%)

Train Epoch: 143 [0/40000 (0%)]	Loss: 0.572852
Train Epoch: 143 [25600/40000 (64%)]	Loss: 0.542377

Test set: Average loss: 0.0059, Accuracy: 7647/9999 (76%)

Train Epoch: 144 [0/40000 (0%)]	Loss: 0.580282
Train Epoch: 144 [25600/40000 (64%)]	Loss: 0.621786

Test set: Average loss: 0.0060, Accuracy: 7647/9999 (76%)

Train Epoch: 145 [0/40000 (0%)]	Loss: 0.643623
Train Epoch: 145 [25600/40000 (64%)]	Loss: 0.568327

Test set: Average loss: 0.0056, Accuracy: 7704/9999 (77%)

Train Epoch: 146 [0/40000 (0%)]	Loss: 0.661468
Train Epoch: 146 [25600/40000 (64%)]	Loss: 0.503757

Test set: Average loss: 0.0056, Accuracy: 7799/9999 (78%)

Train Epoch: 147 [0/40000 (0%)]	Loss: 0.600816

Uploading file best_0.05.pt: 100%|██████████| 100/100 [00:01<00:00, 81.82it/s]


Train Epoch: 154 [0/40000 (0%)]	Loss: 0.649022
Train Epoch: 154 [25600/40000 (64%)]	Loss: 0.516113

Test set: Average loss: 0.0061, Accuracy: 7604/9999 (76%)

Train Epoch: 155 [0/40000 (0%)]	Loss: 0.548216
Train Epoch: 155 [25600/40000 (64%)]	Loss: 0.628664

Test set: Average loss: 0.0056, Accuracy: 7697/9999 (77%)

Train Epoch: 156 [0/40000 (0%)]	Loss: 0.467442
Train Epoch: 156 [25600/40000 (64%)]	Loss: 0.575788

Test set: Average loss: 0.0061, Accuracy: 7642/9999 (76%)

Train Epoch: 157 [0/40000 (0%)]	Loss: 0.427417
Train Epoch: 157 [25600/40000 (64%)]	Loss: 0.534688

Test set: Average loss: 0.0057, Accuracy: 7708/9999 (77%)

Train Epoch: 158 [0/40000 (0%)]	Loss: 0.484518
Train Epoch: 158 [25600/40000 (64%)]	Loss: 0.576748

Test set: Average loss: 0.0057, Accuracy: 7732/9999 (77%)

Train Epoch: 159 [0/40000 (0%)]	Loss: 0.559273
Train Epoch: 159 [25600/40000 (64%)]	Loss: 0.548429

Test set: Average loss: 0.0060, Accuracy: 7639/9999 (76%)

Train Epoch: 160 [0/40000 (0%)]	Loss: 0.530051

Uploading file best_0.05.pt: 100%|██████████| 100/100 [00:01<00:00, 70.86it/s]


Train Epoch: 192 [0/40000 (0%)]	Loss: 0.487367
Train Epoch: 192 [25600/40000 (64%)]	Loss: 0.470443

Test set: Average loss: 0.0057, Accuracy: 7762/9999 (78%)

Train Epoch: 193 [0/40000 (0%)]	Loss: 0.491449
Train Epoch: 193 [25600/40000 (64%)]	Loss: 0.444738

Test set: Average loss: 0.0056, Accuracy: 7805/9999 (78%)

Train Epoch: 194 [0/40000 (0%)]	Loss: 0.509345
Train Epoch: 194 [25600/40000 (64%)]	Loss: 0.619158

Test set: Average loss: 0.0061, Accuracy: 7652/9999 (77%)

Train Epoch: 195 [0/40000 (0%)]	Loss: 0.557623
Train Epoch: 195 [25600/40000 (64%)]	Loss: 0.616175

Test set: Average loss: 0.0055, Accuracy: 7799/9999 (78%)

Train Epoch: 196 [0/40000 (0%)]	Loss: 0.523014
Train Epoch: 196 [25600/40000 (64%)]	Loss: 0.601577

Test set: Average loss: 0.0056, Accuracy: 7737/9999 (77%)

Train Epoch: 197 [0/40000 (0%)]	Loss: 0.580492
Train Epoch: 197 [25600/40000 (64%)]	Loss: 0.602900

Test set: Average loss: 0.0056, Accuracy: 7812/9999 (78%)

Train Epoch: 198 [0/40000 (0%)]	Loss: 0.470305